<a href="https://colab.research.google.com/github/SisopTeam3-SoaTeam5/TP2/blob/main/HPC/Cuaderno_2_Grupo_5_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduccion 
Se eligio hacer un fragmento del problema de n-body, que consiste en calcular la posicion de particulos en base a la fuerzas gravitatorias ejercidad entre ellas. El fragmento elegido consiste en el calculo de la acelaracion de cada particula del sistema en consecuencia de las demas. Se observa en este ejercicio, que ademas de las ventajas de aplicar paralelismo para un gran volumen de datos (las particulas) las desventajas de este si el volumen de datos es pequeño.

Como ya se menciono se utiliza HPC para facilitar la ejecucion de un algoritmo que es sencillo pero conlleva gran volumen de datos.

In [ ]:
#@title ## Armado del ambiente
#@markdown ---
#@markdown ### Especifique la cantidad de particulas en el sistema:
N =  1000#@param {type:"number"}

!pip install pycuda

     |████████████████████████████████| 1.7 MB 5.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 75 kB 3.9 MB/s 
     |████████████████████████████████| 66 kB 4.2 MB/s 
  Created wheel for pycuda: filename=pycuda-2021.1-cp37-cp37m-linux_x86_64.whl size=627661 sha256=de2b74f4ffec68e5cea63cc113b457b403085d6a235ad09325fff841ae06e985
  Stored in directory: /root/.cache/pip/wheels/c4/ef/49/dc6a5feb8d980b37c83d465ecab24949a6aa19458522a9e001
  Created wheel for pytools: filename=pytools-2021.2.9-py2.py3-none-any.whl size=62370 sha256=47c3342df47395fc53e52411ce20edf04c911f8b0149988f028ba6d11dda23a3
  Stored in directory: /root/.cache/pip/wheels/41/b9/6e/94bb014f6484b15ec77e7877f3a227609481ffd98db364504d
Successfully built pycuda pytools


In [ ]:
try:
	from datetime import datetime

	######### Ejecucion secuencial #############

	tiempo_total_cpu = datetime.now()
	import numpy
	import sys
	import pycuda.driver as cuda
	import pycuda.autoinit
	from pycuda.compiler import SourceModule

	tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0 #Funcion para ver el tiempo en ms

	pos  = numpy.random.randn(N,3) #Se define una Matriz de N particulas con la posicion en los 3 ejes (x, y, z) con valores random
	mass=numpy.random.rand(N) #La masa de cada particula N tambien random
 
	G=9.8 #constante de gravedad

	softening=0.1 #variable para corregir el error de presicion q se puede generar

	acc=numpy.array([[0.0,0.0,0.0],
								[0.0,0.0,0.0],
								[0.0,0.0,0.0]])

	N = pos.shape[0];
	a = numpy.zeros((N,3));

	tiempo_particulas_cpu = datetime.now()

#Por cada particula j, calcular la aceleracion que genera en todo el resto de particulas
	for i in range(N):
		for j in range(N):
			dx = pos[j,0] - pos[i,0];
			dy = pos[j,1] - pos[i,1];
			dz = pos[j,2] - pos[i,2];
			inv_r3 = (dx**2 + dy**2 + dz**2 + softening**2)**(-1.5);
			a[i,0] +=  G * (dx * inv_r3) * mass[j];
			a[i,1] +=  G * (dy * inv_r3) * mass[j];
			a[i,2] +=  G * (dz * inv_r3) * mass[j];

	tiempo_particulas_cpu = datetime.now() - tiempo_particulas_cpu

	tiempo_total_cpu = datetime.now() - tiempo_total_cpu

	print('Metricas CPU:')
	print( "Tiempo de medicion de particulas", tiempo_en_ms( tiempo_particulas_cpu   ), "[ms]" )
	print( "Tiempo Total:",                   tiempo_en_ms( tiempo_total_cpu ), "[ms]" )

	print('~~~~~~~~')



	######### Ejecucion paralela #############

	tiempo_total_gpu = datetime.now()



	posCPU= pos

	posCPU = posCPU.astype(numpy.float32())

	massCPU= mass
	massCPU = massCPU.astype(numpy.float32())


	accCPU= numpy.zeros((N,3));
	accCPU = accCPU.astype(numpy.float32())

	accFinal=numpy.empty_like(accCPU)


	posGPU = cuda.mem_alloc( posCPU.nbytes )
	accGPU = cuda.mem_alloc( accCPU.nbytes )
	massGPU = cuda.mem_alloc( massCPU.nbytes )

	cuda.memcpy_htod( posGPU, posCPU )
	cuda.memcpy_htod( accGPU, accCPU )
	cuda.memcpy_htod( massGPU, massCPU )

	module = SourceModule("""
	__global__ void kernel_n_body( int cantidad, float *pos,float G, float *mass,float *acc, float softening)
	{
		int idx = threadIdx.x + blockIdx.x*blockDim.x; //Equivalente a j en ejecucion secuencial
		int idy = threadIdx.y + blockIdx.y*blockDim.y; //Equivalente a i en ejecucion secuencial
		if(idx<cantidad && idy<cantidad){
				float dx= pos[idx*3] - pos[idy*3];
				float dy=pos[idx*3+1]-pos[idy*3+1];
				float dz=pos[idx*3+2]-pos[idy*3+2];
				float inv_r3 = ( powf(dx,2) +  powf(dy,2) +  powf(dz,2) + powf(softening,2));
				inv_r3=powf(inv_r3,-1.5);

				atomicAdd(&acc[idy*3],  G * (dx * inv_r3) * mass[idx]); //Esta funcion se utiliza para que se acumulen todos los resultados de todos los hilos sin problemas
				atomicAdd(&acc[idy*3+1],  G * (dy * inv_r3) * mass[idx]); 
				atomicAdd(&acc[idy*3+2],  G * (dz * inv_r3) * mass[idx]);

		}
	}
	""")

	kernel = module.get_function("kernel_n_body")
	dim_hilo_x = 16
	dim_bloque_x = numpy.int( (pos.shape[0]+dim_hilo_x-1) / dim_hilo_x )
	dim_hilo_y = 16
	dim_bloque_y = numpy.int( (pos.shape[0]+dim_hilo_y-1) / dim_hilo_y)

	tiempo_particulas_gpu = datetime.now()
	kernel( numpy.int32(posCPU.shape[0]) ,posGPU,numpy.float32(G),massGPU,accGPU,numpy.float32(softening),block=( dim_hilo_x, dim_hilo_y, 1 ), grid=(dim_bloque_x, dim_bloque_y,1) )
	tiempo_particulas_gpu = datetime.now() - tiempo_particulas_gpu

	cuda.memcpy_dtoh( accFinal, accGPU )


	tiempo_total_gpu = datetime.now() - tiempo_total_gpu


	print('Metricas GPU:')
	print( "Tiempo de medicion de particulas GPU", tiempo_en_ms( tiempo_particulas_gpu   ), "[ms]" )
	print( "Tiempo Total GPU:",                   tiempo_en_ms( tiempo_total_gpu ), "[ms]" )
	print('-----------------------------------------------------')
	print('Matriz secuencial: ')
	print(a)
	print('-----------------------------------------------------')
	print('Matriz paralelismo: ')
	print(accFinal)
except:
	 print("Ha ocurrido un error en la ejecución. Asegúrese de ejecutar la sección correspondiente al armado del ambiente")

Metricas CPU:
Tiempo de medicion de particulas 6662.812 [ms]
Tiempo Total: 6884.92 [ms]
~~~~~~~~
Metricas GPU:
Tiempo de medicion de particulas GPU 1.736 [ms]
Tiempo Total GPU: 932.738 [ms]
-----------------------------------------------------
Matriz secuencial: 
[[ 255.8221776   252.13606019  879.45344437]
 [-426.85879421 -584.1718609  -436.65765442]
 [-251.86147248  457.35255704 -283.83483183]
 ...
 [-283.70358745 -709.27817855  417.56056736]
 [ 158.65378823  761.04914213  467.69526304]
 [ 770.30435392  841.99740068 -215.44628153]]
-----------------------------------------------------
Matriz paralelismo: 
[[ 255.82227  252.13612  879.4532 ]
 [-426.8586  -584.1718  -436.6576 ]
 [-251.86151  457.35263 -283.8351 ]
 ...
 [-283.7036  -709.2788   417.56067]
 [ 158.65375  761.0488   467.6953 ]
 [ 770.30457  841.9976  -215.44609]]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:96: UserWarning: The CUDA compiler succeeded, but said the following:
nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).



#Conclusiones 
Es inevitable notar que para valores altos en número de partículas el procesamiento en GPU es mucho más corto en tiempo, y al no ser mucho más complejo de desarrollar no existe ninguna desventaja en usarlo. Sin embargo se debe tener en cuenta que para valores bajos, como 2 partículas, el tiempo invertido en copiar los datos a memoria hace que no valga la pena procesar en GPU, ya que siendo tan pocos datos la CPU puede procesarlos rápidamente, y al final el conjunto es más lento en GPU. 

Cabe destacar que solo nos centramos en la parte paralela de calcular las aceleraciones, si se quisiera calcular también la posición en cada instante se debería realizar este proceso para cada momento, y modificar la matriz de posiciones con las aceleraciones obtenidas. En ese caso, al tener que calcular muchas veces la aceleración, las diferencias en tiempo serían mucho mas notorias. Para 5 instantes de tiempo el tiempo ahorrado sería 5 veces el tiempo ahorrado para un instante único como el de eset cuaderno.

#Bibliografia

1.	Waldo Valiente, “Prueba 2 - Imagen - GPU”, 					
  Disponible en: https://github.com/wvaliente/SOA_HPC/blob/main/Ejercicios/Prueba%202%20-%20Imagen%20-%20GPU.ipynb [Accedido: 18/11/2021]

2.	talonmies, “CUDA incrementing global device counter within a kernel”, Disponible en: https://stackoverflow.com/questions/35673928/cuda-incrementing-global-device-counter-within-a-kernel [Accedido: 22/11/2021]

3. John H. Reif and Stephen R. Tate, “Efficient N-body Simulation: Fast Algorithms for Potential Field Evaluation and Trummer’s Problem” Disponible en: https://users.cs.duke.edu/~reif/paper/tate/nbody.pdf [Accedido: 20/11/2021]

4. Alexander G. Gray and Andrew W. Moore , “‘N-Body’ Problems in Statistical Learning”, Disponible en: https://proceedings.neurips.cc/paper/2000/file/7385db9a3f11415bc0e9e2625fae3734-Paper.pdf [Accedido: 20/11/2021]

5. Philip Mocz, “Create Your Own N-body Simulation (With Python)”, Disponible en:  https://medium.com/swlh/create-your-own-n-body-simulation-with-python-f417234885e9 [Accedido: 20/11/2021]